# Naive Bayes Classifier

## Classify iris plants into three species

Le but de ce projet est d'implémenter un **naive Bayes classifier** (from scratch) qui permet de classifier trois types d'iris (fleurs) en fonction de la taille de leur pétales et sépales.

Pour plus de précision sur les data, voir le lien suivant:<br>
https://www.kaggle.com/datasets/uciml/iris

## Librairies

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Loader les data

In [2]:
data = pd.read_csv("./data/iris.csv")

In [3]:
data

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


Nous allons prédire la variable `variety` en fonction des variables `sepal.length`, 	`sepal.width`, `petal.length`, `petal.width`.

## Naive Bayes Classifier

Soient $\mathbf{X} = (X_1 ,\dots, X_P)$ des *variables explicatives*,  $Y$ une *variable réponse* qualitative à valeurs dans $C = \{ c_1, \dots ,c_K \}$. 

Soit également le *train set* $S = \big\{ (\mathbf{x_i}, y_i) \in \mathbb{R}^P \times C :  i = 1, \dots, N \big\}$. Pour tout $k =  1, \dots, K$, on définit $S_k$ comme étant le sous-ensemble du train set $S$ formé des éléments de la classe $c_k$.

Un **Naive Bayes classifier** est un modèle probabiliste de classification qui permet d'estimer la probabilité conditionnelle $p(Y = c_k \mid \mathbf{X} = \mathbf{x})$ que le point $\mathrm{x}$ appartienne à la classe $c_k$, pour tout $k=1,\dots,K$.

En utilisant **hypothèse naïve d’indépendance conditionnelle** ainsi que le **théorème de Bayes**, la *probabilité conditionnelle* et la *prédiction* sont données par les formules suivantes:

<div class="alert alert-block alert-info">
$$
p(c_k  \mid \mathbf{x}) ~\propto~ p(c_k) \prod_{j=1}^{P} p(x_{j} \mid c_k) ~~~\text{ et }~~~ \hat c ~=~ \arg \max_{c_k \in C} \, p(c_k \mid \mathbf{x})
$$
<div>

Dans cette formule, les probabilités $p(c_k)$ et $p(x_{j} \mid c_k)$ doivent être *estimées* à partir des data. Dans ce context, on prendra:

<div class="alert alert-block alert-warning">
$$
p(c_k) = \frac{| S_k |}{N}
$$
<div>

<div class="alert alert-block alert-warning">
$$
p(x_j \mid c_{k}) = \frac{1}{\sqrt {2\pi \sigma_{jk}^2}} \,\exp \left( -\frac{(x_j - \mu_{jk})^2}{2 \sigma_{jk}^2} \right)
$$
où
$$
\mu_{jk} =\frac{1}{| S_k |} \sum_{(\mathbf{x}, y) \in S_k} x_{j} ~~~\text{ et }~~~
\sigma_{jk}^{2} = \frac{1}{(| S_k | - 1)} \sum_{(\mathbf{x}, y) \in S_k} \left(x_{j} - \mu_{jk} \right)^{2}
$$
<div>

La classe `NaiveBayesClassifier` implémente un **Naive Bayes classifier**.

In [4]:
class NaiveBayesClassifier():
    """
    Implements a Naive Bayes Classifier.
    """
    
    def __init__(self):
        """
        Constructor.
        """
        
        self.num_fts = 0
        self.classes = []
        self.class_priors = {}
        self.mu = None
        self.sigma2 = None
        
    
    def initialize(self, X_train, y_train):
        """
        Update the atributes self.num_fts and self.classes 
        based on the train set (X_train, y_train).
        
        Parameters
        ----------
        X_train : ndarray
            features of the train set
        y_train : ndarray
            responses of the train set
        """
        
        self.num_fts = X_train.shape[1]
        try:
            y_train = y_train.flatten()
        except:
            pass
        self.classes = np.unique(y_train)
    
    
    def compute_class_priors(self, X_train, y_train):
        """
        Estimates the class priors $p(c_k)$ from the train set.
        
        Updates the attribute self.class_priors as follows:
        - the keys of the dict are the classes
        - the values of the dict are the corrresponding prior probabilities
        
        Parameters
        ----------
        X_train : ndarray
            features of the train set
        y_train : ndarray
            responses of the train set
        """
        
        nb_elements = X_train.shape[0]
        
        for c in self.classes:
            
            prior = np.sum(y_train==c) / nb_elements
            self.class_priors[c] = prior
    
    
    def compute_fts_distr_params(self, X_train, y_train):
        """
        Estimates the parameters of the feature distributions $p(x_j | c_k)$ 
        from the train set, for each feature $X_j$ and each class $c_k.
        
        Updates the attributes self.mu and self.sigma2 as follows:
        self.mu and self.sigma2 are 2-dim arrays:
        - the (j, k)-th element of self.mu is $\mu_{jk}$
        - the (j, k)-th element of self.sigma2 is $\sigma^2_{jk}$
        
        Parameters
        ----------
        X_train : ndarray
            features of the train set
        y_train : ndarray
            responses of the train set
        """
        
        num_classes = len(self.classes)
        self.mu = np.ones(shape=(self.num_fts, num_classes))
        self.sigma2 = np.ones(shape=(self.num_fts, num_classes))
        
        for k, c in enumerate(self.classes):
            
            mask = y_train.flatten() == c
            X_train_sub = X_train[mask]
            nb_elements = np.sum(mask)
            
            # using numpy functions
            self.mu[:, k] = np.mean(X_train_sub, axis=0)
            self.sigma2[:, k] = np.var(X_train_sub, axis=0)
            
            # implmenting the formula
            #for j in range(self.num_fts):
            #    
            #    self.mu[j, k] = np.sum(X_train_sub[:, j]) / nb_elements
            #    
            #    nb_elements -= 1
            #    self.sigma2[j, k] = np.sum((X_train_sub[:, j] - self.mu[j, k])**2) / nb_elements 

                
    def fit(self, X_train, y_train):
        """
        Fits the naive Bayes classifier on the train set.
        After fitting, the parameters of the classifier are computed, i.e.:
        - the class priors
        - the means and variances of the features' distributions
        
        Parameters
        ----------
        X_train : ndarray
            features of the train set
        y_train : ndarray
            responses of the train set
        """
        
        self.initialize(X_train, y_train)
        self.compute_class_priors(X_train, y_train)
        self.compute_fts_distr_params(X_train, y_train)
        
    
    def Gaussian_density(self, x, mu, sigma2):
        """
        Computes the density of probabilty at x of the Gaussian distribution
        of mean and variance mu and sigma2, respectively.
        
        Parameters
        ----------
        x : float
        mu : float
        sigma2 : float
        
        Returns
        -------
        result : float
        """
        
        num = np.exp( -((x - mu)**2) / (2*sigma2) )
        denom = np.sqrt(2 * np.pi * sigma2)
        result = num / denom
        
        return result
        
    
    def predict_proba(self, X_test):
        """
        Predict the class probabilities of a test set X_test.
        
        Parameters
        ----------
        X_test : ndarray
            Test set
            
        Returns
        -------
        y_proba : ndarray
            Tensor of class probabilities for each element of the test sets.
        """
        
        nb_elements = len(X_test)
        nb_classes = len(self.classes)
        y_proba = np.zeros(shape=(nb_elements, nb_classes))
        
        for e in range(nb_elements):
        
            x = X_test[e, :]
            for k, c in enumerate(self.classes):
                
                p_ck = self.class_priors[c]
                p_xjck = 1.

                for j in range(self.num_fts):
                    mu = self.mu[j,k]
                    sigma2 = self.sigma2[j,k]
                    p_xjck *= self.Gaussian_density(x[j], mu, sigma2)

                p_ckx = p_ck * p_xjck

                y_proba[e, k] = p_ckx
            
        return y_proba
    
    
    def predict(self, X_test):
        """
        Predict the classes of a test set X_test.
        
        Parameters
        ----------
        X_test : ndarray
            Test set
            
        Returns
        -------
        y_pred : ndarray
            Tensor of predictions
        """
        
        y_proba = self.predict_proba(X_test)
        y_pred = np.argmax(y_proba, axis=1)
        y_pred = [nb.classes[i] for i in y_pred]
        
        return y_pred

### Exercice 1

Complétez la méthode `initialize(...)` qui, étant donné un train set `(X_train, y_train)` met à jour les attribut `self.num_fts` et `self.num_classes` qui correspondent au nombre de features et au nombre de classes du train set, respectivement.

Testez votre méthode comme suit:
```
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
print(f"Number of features: {nb.num_fts}\nClasses: {nb.classes}")
```

In [5]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [6]:
nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
print(f"Number of features: {nb.num_fts}\nClasses: {nb.classes}")

Number of features: 4
Classes: ['Setosa' 'Versicolor' 'Virginica']


### Exercice 2

Complétez la méthode `compute_class_priors(...)` qui, étant donné un train set `(X_train, y_train)` met à jour l'attribut `self.class_priors`.  


L'attribut `self.class_priors` est un dictionnaire dont chaque clé est une classe et chaque valeur est la  **probabilités a priori de cette classe (class prior) $p(c_k)$** qui est donnée par la première formule en jaune ci-dessus.

Testez votre méthode comme suit:
```
nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
nb.compute_class_priors(X_train, y_train)
nb.class_priors
```

In [7]:
nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
nb.compute_class_priors(X_train, y_train)
nb.class_priors

{'Setosa': 0.3333333333333333,
 'Versicolor': 0.3416666666666667,
 'Virginica': 0.325}

### Exercice 3

Complétez la méthode `compute_fts_distr_params(...)` qui, étant donné un train set `(X_train, y_train)` met à jour les attributs `self.mu` et `self.sigma2`.

- L'attribut `self.mu` est un 2D-array dont le $(j,k)$-ème élément est $\mu_{jk}$ donné par la seconde formule jaune
- L'attribut `self.sigma2` est un 2D-array dont $(j,k)$-élément est $\sigma^2_{jk}$ donné par la seconde formule jaune

Pour vous éviter de réimplémenter la formule de la moyenne et de la variance, vous pouvez utiliser les instructions `np.mean` et `np.var` (avec l'argument `axis = 0`) appliquées au sous-dataset $S_k$ ($k=1,\dots,3$).

Testez votre méthode comme suit:
```
nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
nb.compute_class_priors(X_train, y_train)
nb.compute_fts_distr_params(X_train, y_train)
print(f"Means and variance of the features' distributions:")
nb.mu, nb.sigma2
```

In [8]:
nb = NaiveBayesClassifier()
nb.initialize(X_train, y_train)
nb.compute_class_priors(X_train, y_train)
nb.compute_fts_distr_params(X_train, y_train)
print(f"Means and variance of the features' distributions:")
nb.mu, nb.sigma2

Means and variance of the features' distributions:


(array([[4.99      , 5.9195122 , 6.53333333],
        [3.4525    , 2.77073171, 2.96666667],
        [1.45      , 4.24146341, 5.52051282],
        [0.245     , 1.32195122, 2.        ]]),
 array([[0.1239    , 0.28693635, 0.4165812 ],
        [0.15249375, 0.10011898, 0.0991453 ],
        [0.033     , 0.22584176, 0.28573307],
        [0.010975  , 0.04122546, 0.08205128]]))

### Exercice 4

Complétez la méthode `fit(...)` qui, étant donné un train set `(X_train, y_train)` applique sucessivement les méthodes:
1. `initialize(...)`
2. `compute_class_priors(...)`
3. `compute_fts_distr_params(...)`

Une fois ces méthodes exécutées, tous les paramètres $p(c_k)$, $\mu_{jk}$ et $\sigma^2_{jk}$ ($j=1,\dots,P$ et $k=1,\dots,K$) de notre classifieur sont calculés.


Testez votre méthode comme suit:
```
nb = NaiveBayesClassifier()
nb.fit(X_train, y_train)

print("Class priors:\n\n", nb.class_priors)
print(f"\nMeans and variances of the features' distributions:")
nb.mu, nb.sigma2
```

In [9]:
nb = NaiveBayesClassifier()
nb.fit(X_train, y_train)

print("Class priors:\n\n", nb.class_priors)
print(f"\nMeans and variances of the features' distributions:")
nb.mu, nb.sigma2

Class priors:

 {'Setosa': 0.3333333333333333, 'Versicolor': 0.3416666666666667, 'Virginica': 0.325}

Means and variances of the features' distributions:


(array([[4.99      , 5.9195122 , 6.53333333],
        [3.4525    , 2.77073171, 2.96666667],
        [1.45      , 4.24146341, 5.52051282],
        [0.245     , 1.32195122, 2.        ]]),
 array([[0.1239    , 0.28693635, 0.4165812 ],
        [0.15249375, 0.10011898, 0.0991453 ],
        [0.033     , 0.22584176, 0.28573307],
        [0.010975  , 0.04122546, 0.08205128]]))

### Exercice 6

Complétez la méthode `compute_Gaussian_density(...)` qui, étant donné un point `x`, une moyenne `mu` est une variance `sigma2` calcule la densité de probabilité Gaussienne suivante:
$$
f(x) = 
\frac{1}{\sqrt{2 \pi \sigma^2}}\; \exp\left(-\frac{\left( x - \mu \right)^2}{2\sigma^2} \right)
$$

### Exercice 7

Complétez la méthode `predict_proba(...)` qui, étant donné un test set `X_test`, calcule les probabilités $p(c_k  \mid \mathbf{x})$ ($k = 1, \dots, K$) selon la formule bleue ci-dessus.

Tester votre méthode comme suit:
```
y_proba = nb.predict_proba(X_test)
```

In [10]:
y_proba = nb.predict_proba(X_test)
# y_proba

### Exercice 8

Complétez la méthode `predict(...)` qui, étant donné un test set `X_test`, calcule les predictions $\hat c$ selon la formule bleue (de droite) ci-dessus. Pour cela, il vous suffit de prendre l'argmax de tableau `predict_proba(X_test)`.

Tester votre méthode comme suit:
```
y_pred = nb.predict(X_test)
print(classification_report(y_pred, y_test))
```

In [11]:
y_pred = nb.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        10
  Versicolor       1.00      1.00      1.00         9
   Virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

